In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [58]:
def NSW(xs):
    return -np.prod(xs*np.array([1,2,40]))

In [53]:
xs = np.array([1/3, 1/3, 1/3])
xs = np.array([1/3, 1/3, 1/3])
NSW(xs)

-0.2962962962962963

In [56]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

In [59]:
res = minimize(NSW, start_pos, method='SLSQP', bounds=bnds ,constraints=cons)
res

     fun: -2.962962870208538
     jac: array([-8.88880694, -8.88536593, -8.89249241])
 message: 'Optimization terminated successfully'
    nfev: 29
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.33327282, 0.33339064, 0.33333654])

In [107]:
alpha = 2
def F(x):
    costs = [1., 2., 3.]
    vals  = np.array([costs[i]*np.power(x[i], 1+1/alpha) for i in range(len(costs))])
    return np.sum(vals/(1+1/alpha))

In [108]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 0.5714285714285716
     jac: array([0.85714287, 0.85714287, 0.85714287])
 message: 'Optimization terminated successfully'
    nfev: 30
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.73469389, 0.18367347, 0.08163265])

In [109]:
def theos(costs):
    den = np.sum([np.power(costs[i], -alpha) for i in range(len(costs))])
    return [np.power(costs[i], -alpha) / den for i in range(len(costs))]

In [110]:
theos([1.,2.,3.])

[0.7346938775510203, 0.18367346938775508, 0.08163265306122448]

# normal tullock

In [154]:
alpha = 1
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(x[i], 1+1/alpha)/vs[i] for i in range(len(vs))])
    return np.sum(vals)

In [155]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 0.16666666666666669
     jac: array([0.33333334, 0.33333334, 0.33333334])
 message: 'Optimization terminated successfully'
    nfev: 29
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.16666666, 0.33333333, 0.5       ])

In [156]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [157]:
theos([1.,2.,3.])

[0.16666666666666666, 0.3333333333333333, 0.5]

In [158]:
# other direction

In [175]:
alpha = 1.001
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(x[i], 1-1/alpha)*vs[i] for i in range(len(vs))])
    return -np.sum(vals)

In [176]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: -5.993941161449936
     jac: array([-0.00598794, -0.00598794, -0.00598794])
 message: 'Optimization terminated successfully'
    nfev: 76
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([0.16653668, 0.33330401, 0.50015931])

In [177]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [178]:
theos([1.,2.,3.])

[0.16653664532144477, 0.33330423948674176, 0.5001591151918134]

In [158]:
# v_i^a

In [182]:
alpha = 2
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(vs[i], alpha)*x[i] for i in range(len(vs))])
    return -np.sum(vals)

In [183]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 1.000000000000015
     jac: array([0.99999999, 4.        , 9.        ])
 message: 'Positive directional derivative for linesearch'
    nfev: 64
     nit: 10
    njev: 6
  status: 8
 success: False
       x: array([1.00000000e+00, 3.78882784e-15, 0.00000000e+00])

In [177]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [178]:
theos([1.,2.,3.])

[0.16653664532144477, 0.33330423948674176, 0.5001591151918134]